Functions and imports


In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
main_copy_dir = os.path.join(parent_dir, "main_copy")
octagon_analysis = os.path.join(parent_dir, "octagon_analysis")
sys.path.append(main_copy_dir)
sys.path.append(octagon_analysis)

print("Added paths to sys.path:", main_copy_dir, octagon_analysis)

Added paths to sys.path: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy /Users/benny/Desktop/MSc/Project/Git/repos/octagon_analysis


Imports with autoreload

In [2]:
%load_ext autoreload
%autoreload 2

import analysis.trajectory_efficiency as trajectory_efficiency
import pandas as pd
import numpy as np
import parse_data.prepare_data as prepare_data
import parse_data.identify_filepaths as identify_filepaths
import analysis.conditioned_player_choice as conditioned_player_choice
import prepare_questionnaire_data
import data_extraction.get_indices as get_indices
import globals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define data folder and eprsonal data file

In [3]:
data_folder = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files'
personal_data_csv = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/personal_data.csv'

Prepare questionnaire data

In [4]:
questionnaire_data_df = prepare_questionnaire_data.preprocess_data()

AQ_scores.csv
GAD_scores.csv
BIS_scores.csv


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny

Get social and solo filenames (solos ordered by pseudonym, neither ordered by date)

In [5]:
social_files, ordered_solos = identify_filepaths.get_filenames(data_folder=data_folder)

Separate first and second solos into lists

In [6]:
first_solos = ordered_solos[::2]
second_solos = ordered_solos[1::2]

Create list of tuples for pseudonyms in socials

In [7]:
import re
pseudo_tuples_list = []
experiment_dates_socials = []
for file in social_files:
     # match the session number and the pseudonym string
    match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?_.*?)_Social\.json', file)
    if match:
        session, date, time, pseudonyms = match.groups()
        pseudo_tuples = pseudonyms.split('_')[0::]
        pseudo_tuples_list.append(pseudo_tuples)
        experiment_dates_socials.append(date)

In [8]:
social_dates = {}
opponent_dict = {}

for i, pseudonym in enumerate(pseudo_tuples_list):
    player0 = pseudonym[0]
    player1 = pseudonym[1]

    if player0 not in social_dates:
        social_dates[player0] = []
        opponent_dict[player0] = []
    if player1 not in social_dates:
        social_dates[player1] = []
        opponent_dict[player1] = []

    social_dates[player0] = experiment_dates_socials[i]
    social_dates[player1] = experiment_dates_socials[i]

    opponent_dict[player0] = player1
    opponent_dict[player1] = player0


Prepare data and get trial lists

In [9]:
df_first_solos, first_solos_lists = prepare_data.prepare_data(data_folder, first_solos, combine=False)
df_second_solos, second_solos_lists = prepare_data.prepare_data(data_folder, second_solos, combine=False)
df_socials, social_lists = prepare_data.prepare_data(data_folder, social_files, combine=False)
df_solos, solos_lists = prepare_data.prepare_data(data_folder, ordered_solos, combine=False)

filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/250417_1/2025-04-17_12-09-51_CY17_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/250417_1/2025-04-17_12-10-56_RL17_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241112_2/2024-11-12_15-13-34_FA12_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241112_2/2024-11-12_15-11-45_SL12_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241210_1/2024-12-10_14-13-47_TE10_FirstSolo.json
Loading complete.
Preprocessing 

In [19]:
len(first_solos)

89

In [10]:
# combined solos
combined_solos = []

for trial_list_idx in range(0, len(solos_lists), 2):

    print(ordered_solos[trial_list_idx], ordered_solos[trial_list_idx + 1]) 

    # concatenate the trial lists for the 2 solos of this session
    trial_list_combined = solos_lists[trial_list_idx] + solos_lists[trial_list_idx + 1]

    combined_solos.append(trial_list_combined)

250417_1/2025-04-17_12-09-51_CY17_FirstSolo.json 250417_1/2025-04-17_12-46-27_CY17_SecondSolo.json
250417_1/2025-04-17_12-10-56_RL17_FirstSolo.json 250417_1/2025-04-17_12-46-05_RL17_SecondSolo.json
241112_2/2024-11-12_15-13-34_FA12_FirstSolo.json 241112_2/2024-11-12_15-44-54_FA12_SecondSolo.json
241112_2/2024-11-12_15-11-45_SL12_FirstSolo.json 241112_2/2024-11-12_15-45-35_SL12_SecondSolo.json
241210_1/2024-12-10_14-13-47_TE10_FirstSolo.json 241210_1/2024-12-10_14-45-41_TE10_SecondSolo.json
241210_1/2024-12-10_14-12-58_TK10_FirstSolo.json 241210_1/2024-12-10_14-45-09_TK10_SecondSolo.json
250509_1/2025-05-09_12-18-03_AM09_FirstSolo.json 250509_1/2025-05-09_12-53-10_AM09_SecondSolo.json
250509_1/2025-05-09_12-18-53_MC09_FirstSolo.json 250509_1/2025-05-09_12-53-00_MC09_SecondSolo.json
250401_2/2025-04-01_15-17-03_AL01_FirstSolo.json 250401_2/2025-04-01_15-52-21_AL01_SecondSolo.json
250401_2/2025-04-01_15-17-26_NL01_FirstSolo.json 250401_2/2025-04-01_15-52-54_NL01_SecondSolo.json
250319_1/2

Choice conditioned on visibility, using conditioned_player_choice.py

Solo-social difference needs matching by pseudonym first

P(H|H first vis)
- first solo 
- second solo 
- combined solo 
- social 
- combined solo - social
- wall sep 1 
- wall sep 2 
- wall sep 4 

P(H|L first vis)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

P(H|opponent vis)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

P(H|opponent not vis)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

P(win)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

Score per unit time
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

- match to pseudonym and store in table


SOLO P(H|H first vis)

filtered to include HighLow trials only

1. First solos
2. Second solos
3. Combined solos

SOCIALS

In [11]:

# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = None
wall_initial_vis_only = False
other_visible = True
inverse_other_visible = False
wall_sep = None
solo = False
data_size_cutoff = 6

In [12]:
## All social datasets
(probabilities3social,
 numerator3social,
  denominator3social) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similari

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

Filter by wall sep

Wall sep: 45°

In [13]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = None
wall_initial_vis_only = False
other_visible = True
inverse_other_visible = False
wall_sep = 1
solo = False
data_size_cutoff = 6

In [14]:
## All social datasets
(probabilities3social_sep1,
 numerator3social_sep1,
  denominator3social_sep1) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repo

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

Wall sep: 90°

In [15]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = None
wall_initial_vis_only = False
other_visible = True
inverse_other_visible = False
wall_sep = 2
solo = False
data_size_cutoff = 6

In [16]:
## All social datasets
(probabilities3social_sep2,
 numerator3social_sep2,
  denominator3social_sep2) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repo

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


...

/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

Wall sep: 180°

In [17]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = None
wall_initial_vis_only = False
other_visible = True
inverse_other_visible = False
wall_sep = 4
solo = False
data_size_cutoff = 6

In [18]:
## All social datasets
(probabilities3social_sep4,
 numerator3social_sep4,
  denominator3social_sep4) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_simi

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


fewer than 6 trials in the denominator, returning np.nan instead of probability


...

/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


fewer than 6 trials in the denominator, returning np.nan instead of probability


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)
/Users/benny/Desktop/MSc/Project/Git/repo

Pseudonym assignments + storing in dictionaries

In [23]:
opp_visible = {}
opp_visible_sep_1 = {}
opp_visible_sep_2 = {}
opp_visible_sep_4 = {}
opp_visible_den = {}
opp_visible_sep_1_den = {}
opp_visible_sep_2_den = {}
opp_visible_sep_4_den = {}

In [25]:
opp_vis_suffix_map = {
    'sep_1': {
        'probability': probabilities3social_sep1,
        'denominator': denominator3social_sep1
    },
    'sep_2': {
        'probability': probabilities3social_sep2,
        'denominator': denominator3social_sep2
    },
    'sep_4': {
        'probability': probabilities3social_sep4,
        'denominator': denominator3social_sep4
    },
    'all_seps': {
        'probability': probabilities3social,
        'denominator': denominator3social
    }
}


In [26]:
opp_vis_dict_map = {
    'sep_1': {
        'probability': opp_visible_sep_1,
        'denominator': opp_visible_sep_1_den
    },
    'sep_2': {
        'probability': opp_visible_sep_2,
        'denominator': opp_visible_sep_2_den
    },
    'sep_4': {
        'probability': opp_visible_sep_4,
        'denominator': opp_visible_sep_4_den
    },
    'all_seps': {
        'probability': opp_visible,
        'denominator': opp_visible_den
    }
}

In [27]:
for i, (player0, player1) in enumerate(pseudo_tuples_list):
    for player in (player0, player1):
        # Initialize if not already
        for suffix_dict in opp_vis_dict_map.values():
            print(suffix_dict)
            for kind in ['probability', 'denominator']:
                if kind not in suffix_dict:
                    suffix_dict[kind] = {}
                if player not in suffix_dict[kind]:
                    suffix_dict[kind][player] = []

    for suffix, data_arrays in opp_vis_suffix_map.items():
        opp_vis_dict_map[suffix]['probability'][player0] = data_arrays['probability'][i, 0]
        opp_vis_dict_map[suffix]['probability'][player1] = data_arrays['probability'][i, 1]
        opp_vis_dict_map[suffix]['denominator'][player0] = len(data_arrays['denominator'][0][i])
        opp_vis_dict_map[suffix]['denominator'][player1] = len(data_arrays['denominator'][1][i])

{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': np.float64(0.9523809523809523), 'RL17': np.float64(0.675)}, 'denominator': {'CY17': 21, 'RL17': 40}}
{'probability': {'CY17': np.float64(0.6666666666666666), 'RL17': np.float64(0.7058823529411765)}, 'denominator': {'CY17': 9, 'RL17': 17}}
{'probability': {'CY17': np.float64(nan), 'RL17': np.float64(0.5555555555555556)}, 'denominator': {'CY17': 5, 'RL17': 18}}
{'probability': {'CY17': np.float64(0.8285714285714286), 'RL17': np.float64(0.6533333333333333)}, 'denominator': {'CY17': 35, 'RL17': 75}}
{'probability': {'CY17': np.float64(0.9523809523809523), 'RL17': np.float64(0.675)

In [31]:
df = pd.read_csv('individual_analyses_table_2.csv', index_col=0)

In [32]:
opp_results = {}

for pseudonym in df.index:
    opp_results[pseudonym] = {
        'pH_opp_visible_sep_1': opp_vis_dict_map['sep_1']['probability'].get(pseudonym, np.nan),
        'pH_opp_visible_sep_2': opp_vis_dict_map['sep_2']['probability'].get(pseudonym, np.nan),
        'pH_opp_visible_sep_4': opp_vis_dict_map['sep_4']['probability'].get(pseudonym, np.nan),
        'pH_opp_visible_all_seps': opp_vis_dict_map['all_seps']['probability'].get(pseudonym, np.nan)
    }

In [33]:
for pseudonym, value in opp_results.items():
    for key in value.keys():
        df.loc[pseudonym, key] = value[key]

In [36]:
df.to_csv('individual_analyses_table_3.csv')

In [38]:
denom_df = pd.read_csv('individual_analyses_trial_counts_2.csv', index_col=0)

In [39]:
denom_results = {}

for pseudonym in denom_df.index:
    denom_results[pseudonym] = {
        'pH_opp_visible_sep_1': opp_vis_dict_map['sep_1']['denominator'].get(pseudonym, np.nan),
        'pH_opp_visible_sep_2': opp_vis_dict_map['sep_2']['denominator'].get(pseudonym, np.nan),
        'pH_opp_visible_sep_4': opp_vis_dict_map['sep_4']['denominator'].get(pseudonym, np.nan),
        'pH_opp_visible_all_seps': opp_vis_dict_map['all_seps']['denominator'].get(pseudonym, np.nan)
    }

In [40]:
for pseudonym, value in denom_results.items():
    for key in value.keys():
        denom_df.loc[pseudonym, key] = value[key]

In [44]:
denom_df.to_csv('individual_analyses_trial_counts_3.csv')